In [1]:
import pandas as pd
import numpy as np
from itertools import repeat
from scipy.sparse import coo_matrix
import scipy
import scipy.signal
from functools import reduce

In [2]:
%time
tl=pd.read_csv('/Users/kristofcoddens/Downloads/M82726bck.csv',skiprows=[1,2,3], low_memory=False, error_bad_lines=False,compression='gzip')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.87 µs


In [3]:
def countnnfails(wfr,df):
    dfc = df[df['Wafer'] == wfr].copy()
    row = np.array(dfc['XPos'])
    col = np.array(dfc['YPos'])
    data = np.array(np.logical_and(dfc['BIN']>349,dfc['BIN']<500))
    data = data.astype(int) #Convert bool to int
    spm = coo_matrix((data, (row, col))).toarray()
    
    kernel = np.array([[1,1,1],[1,0,1],[1,1,1]])
    nncount = scipy.signal.convolve2d(spm, kernel, 'same')
    
    nndfx = np.transpose(np.nonzero(nncount))[:,0]
    nndfy = np.transpose(np.nonzero(nncount))[:,1]
    nndfnncount = np.ravel(nncount)
    nndfnncount = nndfnncount[np.nonzero(nndfnncount)]
    
    nndf = pd.DataFrame({'x' : pd.Series(nndfx),
                         'y' : pd.Series(nndfy),
                         'nncount' : pd.Series(nndfnncount)})

    nndf['Wafer'] = wfr
    
    return(nndf)



In [4]:
tlu=tl[['XPos','YPos','Wafer','BIN']].groupby(['XPos','YPos','Wafer']).head(1)

In [5]:
waferlist=tlu['Wafer'].unique()

In [6]:
%time
sp = list(map(countnnfails,waferlist,repeat(tlu)))

CPU times: user 4 µs, sys: 5 µs, total: 9 µs
Wall time: 26.7 µs


In [7]:
spc = pd.concat(sp)

In [8]:
spc['Wafer'].value_counts()

4     440
3     330
25    302
12    282
2     263
13    239
11    231
6     230
5     229
10    228
14    213
9     212
8     204
7     200
17    193
23    176
22    173
1     165
24    164
19    155
20    149
15    144
21    141
18    127
16    125
Name: Wafer, dtype: int64

In [10]:
spc[np.logical_and(spc['x']==21,spc['y']==25)]

,nncount,x,y,Wafer
16,1,21,25,10
13,1,21,25,11


In [11]:
spc.dtypes

nncount    int64
x          int64
y          int64
Wafer      int64
dtype: object